# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [52]:
import re
import os

In [53]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [54]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [55]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [56]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [57]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[1, 61, 77, 1, 1, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['<UNK>', 'b', 'r', '<UNK>', '<UNK>', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [58]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 50
BATCH_SIZE = 128
EMBED_DIM = 48
HIDDEN_SIZE = 16
MAX_LEN = 50
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 50)
x_test shape: (1987, 50)
trainlabels shape: (6899, 50, 21)
testlabels shape: (1987, 50, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
masking_1 (Masking)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 48)            329952    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 32)            8320      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 21)            693       
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 21)            945      

6209/6209 [==============================] - 24s 4ms/step - loss: 2.9196 - crf_viterbi_accuracy: 0.9436 - val_loss: 3.0596 - val_crf_viterbi_accuracy: 0.8938
Epoch 42/50
6209/6209 [==============================] - 19s 3ms/step - loss: 2.9177 - crf_viterbi_accuracy: 0.9454 - val_loss: 3.0559 - val_crf_viterbi_accuracy: 0.8945
Epoch 43/50
6209/6209 [==============================] - 17s 3ms/step - loss: 2.9172 - crf_viterbi_accuracy: 0.9450 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8987
Epoch 44/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9157 - crf_viterbi_accuracy: 0.9464 - val_loss: 3.0587 - val_crf_viterbi_accuracy: 0.8969
Epoch 45/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9149 - crf_viterbi_accuracy: 0.9485 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8962
Epoch 46/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9139 - crf_viterbi_accuracy: 0.9489 - val_loss: 3.0592 - val_crf_viterbi_accuracy:

In [59]:
# save model
model.save("model/ch_ner_model.h5")

In [60]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            if tag_label != "O": # 当前实体tag之前有其他实体
                result_words.append((start,end,input_data[start: end], tag_label)) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                result_words.append((start,end,input_data[start: end], tag_label)) # 获取实体
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        result_words.append((start,end,input_data[start: end], tag_label)) # 获取结尾的实体
    return result_words

In [61]:
maxlen = 50
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(500):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    result[i+1000] = result_words
    for res in result_words:
        start = res[0]
        end = res[1]
        word = res[2]
        tag = res[3]
        print(i+1000, start, end, "".join(word), tag)

1000 138 142 清热解毒 DRUG_EFFICACY
1000 143 147 化湿除带 DRUG_EFFICACY
1000 148 152 祛瘀止痛 DRUG_EFFICACY
1000 160 165 慢性盆腔炎 DISEASE
1001 36 39 糖尿病 DISEASE
1001 87 91 月经量少 SYMPTOM
1001 95 101 血虚萎黄后错 SYMPTOM
1001 102 106 血虚萎黄 SYMPTOM
1001 107 111 风湿痹痛 SYMPTOM
1001 112 116 肢体麻木 SYMPTOM
1001 116 119 糖尿病 DISEASE
1002 36 42 感冒发热病人 PERSON_GROUP
1002 51 54 高血压 DISEASE
1002 55 58 心脏病 DISEASE_GROUP
1002 59 61 肝病 DISEASE_GROUP
1002 66 68 肾病 DISEASE_GROUP
1002 90 94 月经紊乱 SYMPTOM
1002 109 111 眩晕 SYMPTOM
1002 152 155 过敏者 PERSON_GROUP
1002 158 163 过敏体质者 PERSON_GROUP
1002 192 194 儿童 PERSON_GROUP
1002 242 244 颗粒 DRUG_DOSAGE
1002 245 248 气微香 DRUG_TASTE
1002 249 252 味微苦 DRUG_TASTE
1002 282 286 烘热汗出 SYMPTOM
1002 287 291 头晕耳鸣 SYMPTOM
1002 297 301 五心烦热 SYMPTOM
1002 302 306 腰背酸痛 SYMPTOM
1002 307 311 大便干燥 SYMPTOM
1002 312 315 心烦易 SYMPTOM
1002 387 389 2g SYNDROME
1002 389 391 *1 SYNDROME
1003 55 59 子宫肌瘤 DISEASE
1003 59 63 气滞血瘀 SYNDROME
1003 66 70 经期延长 SYMPTOM
1003 71 75 经量过多 SYMPTOM
1003 76 82 经色紫黯有块 SYMPTOM
1003 83 90

1029 0 2 孕妇 PERSON_GROUP
1029 19 23 肝郁气滞 SYNDROME
1029 31 35 乳腺增生 DISEASE
1030 0 4 补气养血 DRUG_EFFICACY
1030 12 16 气血两虚 SYNDROME
1030 17 21 身体瘦弱 SYMPTOM
1030 22 26 腰膝酸软 SYMPTOM
1030 27 31 月经不调 SYMPTOM
1030 32 35 带下  SYMPTOM
1030 45 48 水蜜丸 DRUG_DOSAGE
1030 49 51 味甜 DRUG_TASTE
1030 52 54 微苦 DRUG_TASTE
1030 55 59   丸剂 DRUG_DOSAGE
1030 60 63 水蜜丸 DRUG_DOSAGE
1030 99 101 止血 DRUG_EFFICACY
1030 432 434 多。 FOOD_GROUP
1030 435 437  尚 FOOD_GROUP
1030 461 463 适量 DRUG_INGREDIENT
1030 464 467 水溶后 DRUG_INGREDIENT
1030 468 470 。  DRUG_INGREDIENT
1030 477 479 业有 DISEASE
1030 489 493 /瓶。  SYMPTOM
1030 528 532 芦、五灵 SYMPTOM
1030 534 538 皂荚或其 SYMPTOM
1030 540 547 。3、感冒时不 SYMPTOM
1030 606 611 阴道不规则 PERSON_GROUP
1030 639 641 咨询 PERSON_GROUP
1030 685 689 状发生改 DRUG_EFFICACY
1030 697 701 0、请将 SYMPTOM
1030 702 706 品放在儿 SYMPTOM
1030 738 740 或药 DRUG_INGREDIENT
1030 741 743 。  DRUG_INGREDIENT
1030 744 747 孕妇禁 DRUG_INGREDIENT
1030 751 753 补气 DRUG_INGREDIENT
1030 764 766 经不 DRUG_INGREDIENT
1030 767 770 、经期 DRUG_INGREDI

1050 45 47 妇女 PERSON_GROUP
1050 47 51 月经量少 SYMPTOM
1050 80 82 颗粒 DRUG_DOSAGE
1050 83 85 味苦 DRUG_TASTE
1050 86 87 涩 DRUG_TASTE
1050 88 90 微酸 DRUG_TASTE
1051 65 70 赤白带下  SYMPTOM
1051 70 74 湿热下注 SYNDROME
1052 15 19 女性患者 PERSON_GROUP
1052 64 66 气香 DRUG_TASTE
1052 67 69 味苦 DRUG_TASTE
1052 70 72 微辛 DRUG_TASTE
1052 155 159 疏肝解郁 DRUG_EFFICACY
1052 160 164 理气止痛 DRUG_EFFICACY
1052 165 169 活血破瘀 DRUG_EFFICACY
1052 175 179 软坚散结 DRUG_EFFICACY
1052 180 184 补气健脾 DRUG_EFFICACY
1052 187 192 乳腺增生  DISEASE
1052 222 226 食欲减退 SYMPTOM
1053 91 96 慢性宫颈炎 DISEASE_GROUP
1053 97 101 宫颈糜烂 SYMPTOM
1053 102 106 阴道炎  DISEASE
1054 21 25   怀化 DRUG_INGREDIENT
1054 79 81 头昏 SYMPTOM
1054 194 198 缩短小鼠 DRUG_EFFICACY
1054 227 231 菌、大肠 SYMPTOM
1054 235 239 型链球菌 DRUG_DOSAGE
1055 26 28 妇女 PERSON_GROUP
1055 28 32 血虚气滞 SYNDROME
1055 33 37 月经不调 SYMPTOM
1055 41 45 经后腹痛 SYMPTOM
1055 45 47 腰痛 SYMPTOM
1055 103 106 味微苦 DRUG_TASTE
1055 136 138    DRUG_DOSAGE
1056 85 89 活血止痛 DRUG_EFFICACY
1056 121 124 软硬下 SYMPTOM
1057 47 49 栓剂 DRUG_DOSAGE

1083 18 22 活血化瘀 DRUG_EFFICACY
1083 23 27 消散乳块 DRUG_EFFICACY
1083 30 34 肝气郁结 SYNDROME
1083 35 39 气滞血瘀 SYNDROME
1083 40 44 乳腺增生 DISEASE
1083 45 49 乳房胀痛 SYMPTOM
1083 49 53 乳腺增生 DISEASE
1084 25 27 孕妇 PERSON_GROUP
1084 64 68 宿有瘕块 SYMPTOM
1084 70 74 血瘀经闭 SYMPTOM
1084 74 78 血瘀经闭 SYMPTOM
1084 79 83 行经腹痛 SYMPTOM
1084 115 119  活血化 DRUG_EFFICACY
1084 129 133 人宿有血 SYMPTOM
1084 137 141 后漏下不 SYMPTOM
1084 142 146 ，胎动不 SYMPTOM
1084 148 152 或血瘀经 SYMPTOM
1084 153 157 ，行经腹 SYMPTOM
1084 160 163 后恶露 SYMPTOM
1085 13 17 止痛调经 DRUG_EFFICACY
1085 20 24 经期腹痛 SYMPTOM
1085 30 34 月经失调 SYMPTOM
1085 37 41 经期腹痛 SYMPTOM
1085 48 54 月经失调   SYMPTOM
1086 1 5 开郁顺气 DRUG_EFFICACY
1086 6 10 调经养血 DRUG_EFFICACY
1086 13 17 月经错后 SYMPTOM
1086 18 22 胸胁胀痛 SYMPTOM
1087 34 37 补气血 DRUG_EFFICACY
1087 38 41 调月经 DRUG_EFFICACY
1087 84 88 活血调经 DRUG_EFFICACY
1087 91 95 气血两虚 SYNDROME
1087 109 115 月经周期错后 SYMPTOM
1087 116 120 行经量少 SYMPTOM
1087 121 125 精神不振 SYMPTOM
1087 126 130 肢体乏力 SYMPTOM
1087 167 171 气血两虚 SYNDROME
1087 174 178 月经不调 SYMPTOM
108

1110 4 6 辛辣 FOOD_GROUP
1110 7 9 生冷 FOOD_GROUP
1110 14 20 感冒发热病人 PERSON_GROUP
1110 28 31 高血压 DISEASE
1110 32 35 心脏病 DISEASE_GROUP
1110 36 38 肝病 DISEASE_GROUP
1110 39 42 糖尿病 DISEASE
1110 43 45 肾病 DISEASE_GROUP
1110 46 49 慢性病 DISEASE_GROUP
1110 64 69 青春期少女 PERSON_GROUP
1110 70 75 更年期妇女 PERSON_GROUP
1110 98 102 月经过少 SYMPTOM
1110 104 108 经期错后 SYMPTOM
1110 110 117 阴道不规则出血 SYMPTOM
1110 171 176 过敏体质者 PERSON_GROUP
1110 204 206 儿童 PERSON_GROUP
1110 238 242 促进造血 DRUG_EFFICACY
1110 243 245 止血 DRUG_EFFICACY
1110 534 538 角叉菜胶 DRUG_EFFICACY
1110 544 548 肿胀及大 SYNDROME
1110 549 553 棉球肉芽 SYMPTOM
1110 554 558 的形成， SYMPTOM
1110 559 563 能抑制羧 SYMPTOM
1110 564 568 基纤维素 SYMPTOM
1110 572 575 渗出液 DRUG_DOSAGE
1110 641 643 一日 DRUG_INGREDIENT
1110 644 646 次。 DRUG_INGREDIENT
1110 647 650  雷允 DRUG_INGREDIENT
1110 654 656 限公 DRUG_INGREDIENT
1110 668 670 品请 DRUG_INGREDIENT
1110 671 674 知医师 DRUG_INGREDIENT
1110 675 677 药师 DRUG_INGREDIENT
1110 678 680 包括 DRUG_INGREDIENT
1110 681 683 何从 DRUG_INGREDIENT
1110 687 689 市或 DR

...

1396 13 16   用 SYMPTOM
1396 17 21 经期腹痛 SYMPTOM
1396 28 34 月经失调   SYMPTOM
1396 34 38 通调气血 DRUG_EFFICACY
1396 39 43 止痛调经 DRUG_EFFICACY
1396 46 50 经期腹痛 SYMPTOM
1396 56 60 月经失调 SYMPTOM
1396 60 62 止痛 DRUG_EFFICACY
1396 62 68 调经通调气血 DRUG_EFFICACY
1396 69 73 止痛调经 DRUG_EFFICACY
1397 63 67 气血两虚 SYNDROME
1397 68 72 气滞血瘀 SYNDROME
1397 87 91 气血两虚 SYNDROME
1397 92 94 气滞 SYNDROME
1397 94 96 血瘀 SYNDROME
1397 99 103 月经不调 SYMPTOM
1397 106 110 月经提前 SYMPTOM
1397 111 115 月经错后 SYMPTOM
1397 116 120 月经量多 SYMPTOM
1397 121 125 神疲乏力 SYMPTOM
1397 126 130 行经腹痛 SYMPTOM
1397 134 137 水蜜丸 DRUG_DOSAGE
1398 19 21    DISEASE
1398 47 50 水蜜丸 DRUG_DOSAGE
1398 77 81 气血两虚 SYNDROME
1398 82 84 气滞 SYNDROME
1398 84 86 血瘀 SYNDROME
1398 89 93 月经不调 SYMPTOM
1398 96 100 月经提前 SYMPTOM
1398 101 105 月经错后 SYMPTOM
1398 105 109 月经提前 SYMPTOM
1398 110 114 月经错后 SYMPTOM
1398 115 119 月经量多 SYMPTOM
1398 120 124 神疲乏力 SYMPTOM
1399 46 50 月经量少 SYMPTOM
1399 54 58 经来腹痛 SYMPTOM
1399 58 62 祛瘀生新 DRUG_EFFICACY
1399 65 69 月经量少 SYMPTOM
1399 73 77 经来腹痛 SYMPTOM

1417 33 37 月经量少 SYMPTOM
1417 41 46 经来腹痛  SYMPTOM
1417 67 69 血瘀 SYNDROME
1417 81 85 月经不调 SYMPTOM
1417 115 117 基本 DISEASE_GROUP
1418 3 7 养血祛瘀 DRUG_EFFICACY
1418 15 18 血热证 SYNDROME
1418 63 67 行而不畅 SYMPTOM
1418 68 72 夹有血块 SYMPTOM
1418 133 137 幅度和频 DRUG_EFFICACY
1418 148 152 产妇产后 DRUG_EFFICACY
1418 202 206 小鼠的红 DRUG_EFFICACY
1418 239 241 抑制 SYNDROME
1419 25 29 活血调经 DRUG_EFFICACY
1419 30 34 补血活血 DRUG_EFFICACY
1419 61 65 新之功能 DRUG_EFFICACY
1419 66 70 川芎化瘀 DRUG_EFFICACY
1419 81 87 、红花能活血 DRUG_EFFICACY
1419 88 93 气，祛瘀生 DRUG_EFFICACY
1419 285 289 育年龄段 SYMPTOM
1419 290 294 妇的疗效 SYMPTOM
1419 386 388 主要 PERSON_GROUP
1419 445 448  口服 DRUG_INGREDIENT
1419 463 467 后服用。 SYMPTOM
1419 468 472 妇在产后 SYMPTOM
1419 472 476 立即服用 SYMPTOM
1419 476 480 ；痛经于 SYMPTOM
1419 481 485 经来潮前 SYMPTOM
1420 13 17 益气养血 DRUG_EFFICACY
1420 18 22 活血调经 DRUG_EFFICACY
1420 25 29 气血两虚 SYNDROME
1420 31 33 血瘀 SYNDROME
1420 36 40 月经不调 SYMPTOM
1420 43 49 月经周期错后 SYMPTOM
1420 50 54 行经量少 SYMPTOM
1420 55 59 精神不振 SYMPTOM
1420 60 64 肢体乏力 SYMP

1440 34 36 血瘀 SYNDROME
1440 39 43 月经不调 SYMPTOM
1440 46 50 经水量少 SYMPTOM
1440 50 52 孕妇 PERSON_GROUP
1440 80 82    DISEASE_GROUP
1440 94 98 祛瘀生新 DRUG_EFFICACY
1440 101 105 月经量少 SYMPTOM
1440 109 113 经来腹痛 SYMPTOM
1441 6 10 活血调经 DRUG_EFFICACY
1441 13 17 气血两虚 SYNDROME
1441 31 37 月经周期错后 SYMPTOM
1441 38 42 行经量少 SYMPTOM
1441 43 47 精神不振 SYMPTOM
1441 57 61 月经不调 SYMPTOM
1441 61 67 血瘀气血两虚 SYNDROME
1441 112 114 师或 DISEASE_GROUP
1441 145 149 尚不明确 SYNDROME
1441 152 156 吉林省正 SYMPTOM
1441 159 165 业有限公司  SYMPTOM
1441 166 170 口服。一 SYMPTOM
1441 171 175 6克，一 SYMPTOM
1441 176 180 2次。  SYMPTOM
1441 180 182  用 SYMPTOM
1441 182 185 于气血 DRUG_EFFICACY
1441 186 189 虚所致 DRUG_EFFICACY
1442 43 45 止痛 DRUG_EFFICACY
1442 48 52 月经不调 SYMPTOM
1442 53 57 经期腹痛 SYMPTOM
1442 57 59 孕妇 PERSON_GROUP
1442 76 80 月经不调 SYMPTOM
1442 81 85 经期腹痛 SYMPTOM
1442 85 87    PERSON_GROUP
1442 87 89 孕妇 PERSON_GROUP
1443 59 61 孕妇 PERSON_GROUP
1443 61 63 孕妇 PERSON_GROUP
1443 76 78 孕妇 PERSON_GROUP
1443 97 101 月经量少 SYMPTOM
1443 116 118    DISEASE_GRO

1461 43 45    DISEASE
1461 68 72 开郁顺气 DRUG_EFFICACY
1461 73 77 调经养血 DRUG_EFFICACY
1461 80 84 月经错后 SYMPTOM
1462 14 18 益气养血 DRUG_EFFICACY
1462 19 23 活血调经 DRUG_EFFICACY
1462 26 30 气血两虚 SYNDROME
1462 46 50 月经不调 SYMPTOM
1462 53 59 月经周期错后 SYMPTOM
1462 60 64 行经量少 SYMPTOM
1462 65 69 精神不振 SYMPTOM
1462 70 74 肢体乏力 SYMPTOM
1462 124 127 补气血 DRUG_EFFICACY
1462 128 131 调月经 DRUG_EFFICACY
1462 169 173 月经不调 SYMPTOM
1462 176 182 月经周期错后 SYMPTOM
1463 47 51 月经量少 SYMPTOM
1463 52 57 产后腹痛  SYMPTOM
1463 79 83 活血调经 DRUG_EFFICACY
1464 37 44 清热除湿 止带 DRUG_EFFICACY
1464 44 47 清热  DRUG_EFFICACY
1464 78 82 湿热下注 SYNDROME
1464 83 86 赤白带 SYMPTOM
1465 29 31 孕妇 PERSON_GROUP
1465 34 36    DISEASE_GROUP
1465 81 85 月经量少 SYMPTOM
1465 89 93 经来腹痛 SYMPTOM
1465 128 130 血瘀 SYNDROME
1465 140 144 经水量少 SYMPTOM
1466 17 20 补气血 DRUG_EFFICACY
1466 21 24 调月经 DRUG_EFFICACY
1466 27 31 头晕心慌 SYMPTOM
1466 32 36 疲乏无力 SYMPTOM
1466 37 41 月经量少 SYMPTOM
1466 45 51 经期后错头晕 SYMPTOM
1466 54 57 补气血 DRUG_EFFICACY
1466 58 61 调月经 DRUG_EFFICACY
1466 64 68 月经不

1482 11 15 眩晕耳鸣 SYMPTOM
1482 16 20 烦躁失眠 SYMPTOM
1482 121 123 s; FOOD_GROUP
1482 126 128    FOOD_GROUP
1482 131 133 食辛 DISEASE
1482 143 147 冒时不宜 SYMPTOM
1482 154 157 月经紊 DISEASE
1482 158 161 或其他 DISEASE_GROUP
1482 162 165 病如： DISEASE
1482 166 171 血压、心脏 DISEASE_GROUP
1482 172 175 、糖尿 DISEASE
1482 191 194 者，应 DISEASE
1482 250 253 本品不 PERSON_GROUP
1482 256 261 服用。6、 PERSON_GROUP
1482 288 290 时禁 PERSON_GROUP
1482 290 297 止使用。8、请 PERSON_GROUP
1482 300 305 放在儿童过 PERSON_GROUP
1482 332 334 时禁 PERSON_GROUP
1482 382 386   长春 DRUG_EFFICACY
1482 387 391 平药业有 DRUG_EFFICACY
1482 402 406 烦安神。 SYMPTOM
1482 407 415 于更年期潮热汗出 SYMPTOM
1482 415 419 ，眩晕耳 DRUG_EFFICACY
1482 420 424 ，烦躁失 DRUG_EFFICACY
1483 84 88 。  清 SYMPTOM
1484 17 21 气血两虚 SYNDROME
1484 35 41 月经周期错后 SYMPTOM
1484 42 46 行经量少 SYMPTOM
1484 47 51 精神不振 SYMPTOM
1484 52 56 肢体乏力 SYMPTOM
1484 56 61 行经量少  SYMPTOM
1484 120 124   用于 SYMPTOM
1484 127 133 虚所致的月经 SYMPTOM
1484 163 166 明确。 DRUG_EFFICACY
1484 167 170  每丸 DRUG_EFFICACY
1485 8 12 活血化瘀 DRUG_EFFICA

In [51]:
result

{1000: [(123, 135, '50毫升装）-安徽天洋药', 'DRUG_EFFICACY'),
  (136, 140, '  清热', 'DRUG_EFFICACY'),
  (141, 145, '毒，化湿', 'DRUG_EFFICACY')],
 1001: [(31, 33, '孕妇', 'PERSON_GROUP'),
  (79, 81, '活血', 'DRUG_EFFICACY'),
  (82, 84, '通络', 'DRUG_INGREDIENT'),
  (87, 91, '月经量少', 'SYMPTOM'),
  (95, 101, '血虚萎黄后错', 'SYMPTOM'),
  (102, 106, '血虚萎黄', 'SYMPTOM'),
  (107, 111, '风湿痹痛', 'SYMPTOM'),
  (112, 116, '肢体麻木', 'SYMPTOM'),
  (116, 119, '糖尿病', 'DISEASE')],
 1002: [(36, 38, '感冒', 'DISEASE'),
  (38, 40, '发热', 'SYMPTOM'),
  (51, 54, '高血压', 'DISEASE'),
  (55, 58, '心脏病', 'DISEASE_GROUP'),
  (59, 61, '肝病', 'DISEASE_GROUP'),
  (62, 65, '糖尿病', 'DISEASE'),
  (66, 68, '肾病', 'DISEASE_GROUP'),
  (69, 72, '慢性病', 'DISEASE_GROUP'),
  (90, 94, '月经紊乱', 'SYMPTOM'),
  (109, 111, '眩晕', 'SYMPTOM'),
  (152, 155, '过敏者', 'PERSON_GROUP'),
  (192, 194, '儿童', 'PERSON_GROUP'),
  (242, 244, '颗粒', 'DRUG_DOSAGE'),
  (245, 248, '气微香', 'DRUG_TASTE'),
  (249, 252, '味微苦', 'DRUG_TASTE'),
  (253, 259, '  滋养肝肾', 'DRUG_EFFICACY'),
  (260, 264,

In [24]:
from keras.models import load_model
import numpy as np

maxlen = 40
sentence = " 北京同仁堂科技发展股份有限公司制药厂  1.忌食辛辣，少进油腻。 2.感冒发热病人不宜服用。 3.有高血压、心脏病、肝病、糖尿病、肾病等慢性病严重者应在医师指导下服用。 4.伴有月经紊乱者，应在医师指导下服用。 5.眩晕症状较重者，应及时去医院就诊。 6.服药2周症状无缓解，应去医院就诊。 7.对本品过敏者禁用，过敏体质者慎用。 8.本品性状发生改变时禁止使用。 9.请将本品放在儿童不能接触的地方。 10.如正在使用其他药品，使用本品前请咨询医师或药师。  本品为浅黄色至棕黄色颗粒，气微香，味微苦。  滋养肝肾、宁心安神。用于更年期综合症属阴虚肝旺症，症见烘热汗出，头晕耳鸣，失眠多梦，五心烦热，腰背酸痛，大便干燥，心烦易怒，舌红少苔，脉弦细或弦细 开水冲服。一次1袋(12g)，一日3次。  如与其他药物同时使用可能会发生药物相互作用，详情请咨询医师或药师。  12g*10袋/盒  用于更年期综合症属阴虚肝旺症  铝塑复合膜包装，每袋装12克，每盒装10袋。  非处方药物（甲类）,中药保护品种二级  12g*10袋/盒  用于更年期综合症属阴虚肝旺更年期综合症气微香，味微苦。"
#sentence = sentence.replace(' ','_')
sentences = sentence.split('。')
y_ner = []

for sent in sentences:
    sent = sent.replace(' ','_')
    sent_chars = list(sent+'。')
    sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

    sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
    y_pred = model.predict(sent2id_new)
    y_label = np.argmax(y_pred, axis=2)
    y_label = y_label.reshape(1, -1)[0]
    y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
    y_ner.extend(y_ner_)

In [27]:
result_words = get_valid_nertag(sentence, y_ner)
for (word, tag) in result_words:
    print("".join(word), tag)

感冒 DISEASE
发热 SYMPTOM
高血压 DISEASE
心脏病 DISEASE_GROUP
肝病 DISEASE_GROUP
糖尿病 DISEASE
肾病 DISEASE_GROUP
慢性病 DISEASE_GROUP
月经紊乱 SYMPTOM
眩晕 SYMPTOM
过敏者 PERSON_GROUP
儿童 PERSON_GROUP
颗粒 DRUG_DOSAGE
气微香 DRUG_TASTE
味微苦 DRUG_TASTE
  滋养肝肾 DRUG_EFFICACY
宁心安神 DRUG_EFFICACY
烘热汗出 SYMPTOM
头晕耳鸣 SYMPTOM
失眠多梦 SYMPTOM
五心烦热 SYMPTOM
腰背酸 SYMPTOM


In [25]:
list(zip(list(sentence),y_ner))

[(' ', 'O'),
 ('北', 'O'),
 ('京', 'O'),
 ('同', 'O'),
 ('仁', 'O'),
 ('堂', 'O'),
 ('科', 'O'),
 ('技', 'O'),
 ('发', 'O'),
 ('展', 'O'),
 ('股', 'O'),
 ('份', 'O'),
 ('有', 'O'),
 ('限', 'O'),
 ('公', 'O'),
 ('司', 'O'),
 ('制', 'O'),
 ('药', 'O'),
 ('厂', 'O'),
 (' ', 'O'),
 (' ', 'O'),
 ('1', 'O'),
 ('.', 'O'),
 ('忌', 'O'),
 ('食', 'O'),
 ('辛', 'O'),
 ('辣', 'O'),
 ('，', 'O'),
 ('少', 'O'),
 ('进', 'O'),
 ('油', 'O'),
 ('腻', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('2', 'O'),
 ('.', 'O'),
 ('感', 'B-DISEASE'),
 ('冒', 'I-DISEASE'),
 ('发', 'B-SYMPTOM'),
 ('热', 'I-SYMPTOM'),
 ('病', 'O'),
 ('人', 'O'),
 ('不', 'O'),
 ('宜', 'O'),
 ('服', 'O'),
 ('用', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('3', 'O'),
 ('.', 'O'),
 ('有', 'O'),
 ('高', 'B-DISEASE'),
 ('血', 'I-DISEASE'),
 ('压', 'I-DISEASE'),
 ('、', 'O'),
 ('心', 'B-DISEASE_GROUP'),
 ('脏', 'I-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('肝', 'B-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('糖', 'B-DISEASE'),
 ('尿', 'I-DISEASE'),
 ('病', 'I-DISEASE'),
 ('、'